In [1]:
import sys
import os
import time
import requests
import re
import itertools
import string
import math
import concurrent.futures

In [3]:
available_names = []
invalid_names = []
regex = re.compile(r'[^a-zA-Z0-9_.]')
client = requests.Session()
rate_limited = False
start_time = time.time()

`Example`: 

Generate all 4 character strings with 2 or less syllables, and check their availability.

In [2]:
def count_syllables(word):
    # Regex pattern to match syllables
    pattern = r'([aeiouyAEIOUY]+[^e.\s])|([aiouyAEIOUY]+\b)|(\b[^aeiouy0-9.]+e\b)'
    # Find all non-overlapping matches of the regex pattern in the word
    matches = re.findall(pattern, word)
    # The number of syllables is the number of matches
    return len(matches)

username_list = [''.join(combination) for combination in itertools.product(string.ascii_lowercase, repeat=4)]

# filter the ones with 2 syllables or less
username_list = [username for username in username_list if count_syllables(username) <= 2]

In [4]:
def check_username(username):
    retry = True
    while retry:
        retry = False
        result = bool(regex.search(username))
        if not (result or (len(username) < 3) or (len(username) > 16)):
            res = client.get('https://api.mojang.com/users/profiles/minecraft/' + username)
            if res.status_code == 200:
                pass
            
            elif res.status_code == 404:
                print(f'{username}')
                # from stackoverflow:
                # concurrent.futures.ThreadPoolExecutor allow only one thread to access the common data structure or location in memory at a time; the threading.Lock() primitive is used to manage this, so race conditions don't occur!
                available_names.append(username)

            elif res.status_code == 429:
                end_time = time.time()
                global start_time
                time_to_wait = math.ceil(200 - (end_time - start_time))
                global rate_limited
                if not rate_limited:
                    rate_limited = True
                    print(f'Request is being refused due to IP being rate limited. Waiting {time_to_wait} seconds before reattempting...')
                retry = True
                time.sleep(time_to_wait)
                rate_limited = False
                start_time = time.time()

            else:
                res.raise_for_status()
                print(f'Unhandled HTTP status code: {res.status_code}. Exiting...')
                sys.exit()
                
        else:
            print(f'{username} is an invalid username.')
            invalid_names.append(username)

In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    try:
        executor.map(check_username, username_list)
    except Exception as exc:
        print(f'There is a problem: {exc}. Exiting...')
        sys.exit()
print()
print(f'Available username(s): {available_names}')
if invalid_names:
    print(f'Invalid username(s): {invalid_names}')